# Visualización Global de Carreras y Ofertas Laborales con UMAP
Este notebook compara los vectores académicos de 23 carreras con los vectores de ofertas laborales, aplicando UMAP para reducción de dimensionalidad y generando una visualización interactiva en HTML.

In [1]:
# 1. Importar librerías y cargar datos procesados
import pickle
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.io as pio
import matplotlib.colors as mcolors
import umap
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

# Cargar datos procesados
with open('datos_procesados.pkl', 'rb') as f:
    datos = pickle.load(f)

habilidades = datos['habilidades']
grupos_bge_ngram = datos['grupos_bge_ngram']
tfidf_epn_69d = datos['tfidf_epn_69d']

print("✓ Datos cargados correctamente")
print(f"Carreras académicas disponibles: {list(tfidf_epn_69d.columns)}")

c:\Users\diego\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✓ Datos cargados correctamente
Carreras académicas disponibles: ['Licenciatura Administracion De Empresas', 'Ingenieria Agroindustria', 'Ingenieria Ambiental', 'Ciencias De Datos E Inteligencia Artificial', 'Ingenieria En Ciencias De La Computacion', 'Economia', 'Ingenieria En Electricidad', 'Ingenieria En Electronica Y Automatizacion', 'Fisica', 'Ingenieria En Geologia', 'Ingenieria De La Produccion', 'Matematica', 'Matematica Aplicada', 'Ingenieria En Materiales', 'Ingenieria En Mecanica', 'Ingenieria En Mecatronica', 'Ingenieria En Petroleos', 'Ingenieria Quimica', 'Ingenieria En Sistemas De Informacion', 'Ingenieria En Software', 'Ingenieria En Telecomunicacion De La Informacion', 'Ingenieria En Telecomunicaciones', 'Ingenieria Civil', 'Seguridad De Redes De Informacion']


In [2]:
# 2. Definir carreras a analizar y cargar ofertas laborales
# Formato: (nombre_académico, nombre_visualización, ruta_csv)
CARRERAS_CONFIG = [
    ('Ingenieria En Ciencias De La Computacion', 'Computación', 'todas_las_plataformas/Computación/Computación_Merged.csv'),
    ('Ciencias De Datos E Inteligencia Artificial', 'Ciencias De Datos E IA', 'todas_las_plataformas/Ciencia_de_Datos/Ciencia_de_Datos_Merged.csv'),
    ('Ingenieria En Software', 'Software', 'todas_las_plataformas/Software/Software_Merged.csv'),
    ('Ingenieria En Sistemas De Informacion', 'Sistemas de Información', 'todas_las_plataformas/Sistemas_de_Información/Sistemas_de_Información_Merged.csv'),
    ('Licenciatura Administracion De Empresas', 'Administración de Empresas', 'todas_las_plataformas/Administración_de_Empresas/Administración_de_Empresas_Merged.csv'),
    ('Ingenieria Agroindustria', 'Agroindustria', 'todas_las_plataformas/Agroindustria/Agroindustria_Merged.csv'),
    ('Matematica', 'Matemática', 'todas_las_plataformas/Matemática/Matemática_Merged.csv'),
    ('Matematica Aplicada', 'Matemática Aplicada', 'todas_las_plataformas/Matemática_Aplicada/Matemática_Aplicada_Merged.csv'),
    ('Fisica', 'Física', 'todas_las_plataformas/Física/Física_Merged.csv'),
    ('Ingenieria En Geologia', 'Geología', 'todas_las_plataformas/Geología/Geología_Merged.csv'),
    ('Ingenieria De La Produccion', 'Ingeniería De La Producción', 'todas_las_plataformas/Ingeniería_de_la_Producción/Ingeniería_de_la_Producción_Merged.csv'),
    ('Ingenieria En Materiales', 'Materiales', 'todas_las_plataformas/Materiales/Materiales_Merged.csv'),
    ('Ingenieria En Mecanica', 'Mecánica', 'todas_las_plataformas/Mecánica/Mecánica_Merged.csv'),
    ('Ingenieria En Mecatronica', 'Mecatrónica', 'todas_las_plataformas/Mecatrónica/Mecatrónica_Merged.csv'),
    ('Ingenieria En Petroleos', 'Petróleos', 'todas_las_plataformas/Petróleos/Petróleos_Merged.csv'),
    ('Ingenieria Quimica', 'Ingeniería Química', 'todas_las_plataformas/Ingeniería_Química/Ingeniería_Química_Merged.csv'),
    ('Ingenieria En Telecomunicaciones', 'Telecomunicaciones', 'todas_las_plataformas/Telecomunicaciones/Telecomunicaciones_Merged.csv'),
    ('Ingenieria Civil', 'Ingeniería Civil', 'todas_las_plataformas/Ingeniería_Civil/Ingeniería_Civil_Merged.csv'),
    ('Economia', 'Economía', 'todas_las_plataformas/Economía/Economía_Merged.csv'),
    ('Ingenieria En Electricidad', 'Electricidad', 'todas_las_plataformas/Electricidad/Electricidad_Merged.csv'),
    ('Ingenieria En Electronica Y Automatizacion', 'Electrónica Y Automatización', 'todas_las_plataformas/Electrónica_y_Automatización/Electrónica_y_Automatización_Merged.csv'),
    ('Ingenieria Ambiental', 'Ingeniería Ambiental', 'todas_las_plataformas/Ingeniería_Ambiental/Ingeniería_Ambiental_Merged.csv'),
]
print(f"Se analizarán {len(CARRERAS_CONFIG)} carreras")

Se analizarán 22 carreras


In [9]:
# 3. Procesar ofertas laborales para todas las carreras

def procesar_ofertas_carrera(ruta_csv):
    try:
        df = pd.read_csv(ruta_csv, dtype=str)
        textos = df[['skills','description']].fillna('').agg(' '.join, axis=1).str.lower().tolist()
        fechas = pd.to_datetime(df.get('date_posted_norm'), errors='coerce')
        vectorizer = CountVectorizer(vocabulary=habilidades, analyzer='word', ngram_range=(1, 5), lowercase=True)
        X = vectorizer.transform(textos)
        matriz_td = pd.DataFrame(X.T.toarray(), index=vectorizer.get_feature_names_out())
        matriz_69d = pd.DataFrame(0, index=grupos_bge_ngram.keys(), columns=range(len(textos)))
        for label, terms in grupos_bge_ngram.items():
            matriz_69d.loc[label] = matriz_td.loc[terms].sum(axis=0)
        matriz_69d.index = [', '.join(grupos_bge_ngram[label][:3]) + (' ...' if len(grupos_bge_ngram[label])>3 else '') for label in grupos_bge_ngram.keys()]
        tfidf_transformer = TfidfTransformer(norm='l2').fit(matriz_69d.values)
        tfidf_69d = pd.DataFrame(tfidf_transformer.transform(matriz_69d.values).toarray(), index=matriz_69d.index, columns=matriz_69d.columns).T
        return tfidf_69d, fechas
    except Exception as e:
        print(f"❌ Error procesando {ruta_csv}: {e}")
        return None, None

# Helpers de color
import colorsys
from datetime import datetime

def saturate_color(color, sat_target=0.9):
    r, g, b = mcolors.to_rgb(color)
    h, l, s = colorsys.rgb_to_hls(r, g, b)
    r2, g2, b2 = colorsys.hls_to_rgb(h, l, sat_target)
    return mcolors.to_hex((r2, g2, b2))

def colors_by_date_intensity(base_color, fechas_series):
    # Manejar lista/Index/Series y NaTs de forma robusta
    if fechas_series is None or (hasattr(fechas_series, '__len__') and len(fechas_series) == 0):
        return [], []
    now = pd.Timestamp(datetime.now().date())
    fechas = pd.Series(pd.to_datetime(fechas_series, errors='coerce'))
    deltas = now - fechas
    dias = deltas.dt.days
    # Si todas son NaT, dias.max() será NaN; usar 1 para evitar división por cero
    dias = dias.fillna(dias.max())
    max_days = int(dias.max()) if pd.notna(dias.max()) and dias.max() > 0 else 1
    pesos = 1 - (dias / max_days)
    r, g, b = mcolors.to_rgb(base_color)
    h, l, s0 = colorsys.rgb_to_hls(r, g, b)
    colores = []
    for w in pesos:
        w = float(w) if pd.notna(w) else 0.0
        s = 0.25 + 0.75 * w
        r2, g2, b2 = colorsys.hls_to_rgb(h, l, s)
        a = 0.35 + 0.65 * w
        colores.append(f"rgba({int(r2*255)},{int(g2*255)},{int(b2*255)},{a:.3f})")
    return colores, pesos

vectores_academicos = []
vectores_laborales = []
nombres_carreras = []

print("Procesando ofertas laborales...\n")
for nombre_acad, nombre_vis, ruta_csv in CARRERAS_CONFIG:
    if nombre_acad not in tfidf_epn_69d.columns:
        print(f"⚠ '{nombre_acad}' no encontrado en datos académicos, se omite")
        continue
    tfidf_ofertas, fechas = procesar_ofertas_carrera(ruta_csv)
    if tfidf_ofertas is None:
        continue
    vector_acad = tfidf_epn_69d.T.loc[[nombre_acad]]
    vectores_academicos.append((nombre_vis, vector_acad))
    vectores_laborales.append((nombre_vis, tfidf_ofertas, fechas))
    nombres_carreras.append(nombre_vis)
    print(f"✓ {nombre_vis}: {len(tfidf_ofertas)} ofertas")
print(f"\n✓ Procesadas {len(nombres_carreras)} carreras correctamente")

Procesando ofertas laborales...

✓ Computación: 4101 ofertas
✓ Ciencias De Datos E IA: 4836 ofertas
✓ Software: 5873 ofertas
✓ Sistemas de Información: 6157 ofertas
✓ Administración de Empresas: 5220 ofertas
✓ Agroindustria: 1688 ofertas
✓ Matemática: 399 ofertas
✓ Matemática Aplicada: 383 ofertas
✓ Física: 1614 ofertas
✓ Geología: 1074 ofertas
✓ Ingeniería De La Producción: 4503 ofertas
✓ Materiales: 2127 ofertas
✓ Mecánica: 1633 ofertas
✓ Mecatrónica: 1338 ofertas
✓ Petróleos: 1551 ofertas
✓ Ingeniería Química: 1148 ofertas
✓ Telecomunicaciones: 2935 ofertas
✓ Ingeniería Civil: 3864 ofertas
✓ Economía: 2506 ofertas
✓ Electricidad: 2066 ofertas
✓ Electrónica Y Automatización: 6349 ofertas
✓ Ingeniería Ambiental: 3295 ofertas

✓ Procesadas 22 carreras correctamente


In [10]:
# 4. Calcular UMAP global (una sola vez para todas las carreras)
colores_base = list(mcolors.TABLEAU_COLORS.values()) + list(mcolors.CSS4_COLORS.values())

CAREER_LABELS = {
    'Licenciatura Administracion De Empresas': 'ADMINISTRACIÓN DE EMPRESAS',
    'Ingenieria Agroindustria': 'AGROINDUSTRIA',
    'Ciencias De Datos E Inteligencia Artificial': 'CIENCIA DE DATOS E INTELIGENCIA ARTIFICIAL',
    'Ingenieria En Ciencias De La Computacion': 'COMPUTACIÓN',
    'Economia': 'ECONOMÍA',
    'Ingenieria En Electricidad': 'ELECTRICIDAD',
    'Ingenieria En Electronica Y Automatizacion': 'ELECTRÓNICA Y AUTOMATIZACIÓN',
    'Fisica': 'FÍSICA',
    'Ingenieria En Geologia': 'GEOLOGÍA',
    'Ingenieria Ambiental': 'INGENIERÍA AMBIENTAL',
    'Ingenieria Civil': 'INGENIERÍA CIVIL',
    'Ingenieria De La Produccion': 'INGENIERÍA DE LA PRODUCCIÓN',
    'Ingenieria Quimica': 'INGENIERÍA QUÍMICA',
    'Matematica': 'MATEMÁTICA',
    'Matematica Aplicada': 'MATEMÁTICA APLICADA',
    'Ingenieria En Materiales': 'MATERIALES',
    'Ingenieria En Mecanica': 'MECÁNICA',
    'Ingenieria En Mecatronica': 'MECATRÓNICA',
    'Ingenieria En Petroleos': 'PETRÓLEOS',
    'Ingenieria En Sistemas De Informacion': 'SISTEMAS DE INFORMACIÓN',
    'Ingenieria En Software': 'SOFTWARE',
    'Ingenieria En Tecnologías De La Información': 'TECNOLOGÍAS DE LA INFORMACIÓN',
    'Ingenieria En Telecomunicaciones': 'TELECOMUNICACIONES',
    'Ingenieria En Telecomunicacion De La Informacion': 'TECNOLOGÍAS DE LA INFORMACIÓN'
}
COLOR_OVERRIDES = {'Ingenieria Quimica': '#E74C3C'}

# Enriquecer y ordenar vectores
temp = []
for vis, vec_acad in vectores_academicos:
    match = next(vl for vl in vectores_laborales if vl[0] == vis)
    tfidf_lab_69d, fechas = match[1], match[2]

    display = CAREER_LABELS.get(vis, vis.upper())
    temp.append((display, vis, vec_acad, tfidf_lab_69d, fechas))

temp.sort(key=lambda x: x[0])

vectores_academicos_sorted = []
vectores_laborales_sorted = []
for display, acad, vec_acad, tfidf_lab_69d, fechas in temp:
    vectores_academicos_sorted.append((display, vec_acad, acad))
    vectores_laborales_sorted.append((display, tfidf_lab_69d, fechas, acad))

nombres_carreras = [t[0] for t in temp]

df_academicos = []
df_laborales = []
labels_academicos = []
labels_laborales = []
colores_academicos = []
colores_laborales = []
fechas_laborales = []

for i, (display, vector_acad, acad_name) in enumerate(vectores_academicos_sorted):
    if hasattr(vector_acad, 'values'):
        df_academicos.append(vector_acad.values[0])
    else:
        df_academicos.append(vector_acad)
    labels_academicos.append(display)
    base = COLOR_OVERRIDES.get(acad_name, colores_base[i % len(colores_base)])
    colores_academicos.append(saturate_color(base, sat_target=0.95))

for i, (display, tfidf_lab_69d, fechas, acad_name) in enumerate(vectores_laborales_sorted):
    base = COLOR_OVERRIDES.get(acad_name, colores_base[i % len(colores_base)])
    base_sat = saturate_color(base, sat_target=0.9)
    if hasattr(tfidf_lab_69d, 'iloc'):
        for j in range(tfidf_lab_69d.shape[0]):
            df_laborales.append(tfidf_lab_69d.iloc[j].values)
            labels_laborales.append(display)
            colores_laborales.append(mcolors.to_rgba(base_sat, alpha=0.9))
        if fechas is not None:
            fechas_laborales.extend(list(fechas))
        else:
            fechas_laborales.extend([pd.NaT] * tfidf_lab_69d.shape[0])
    else:
        for j in range(tfidf_lab_69d.shape[0]):
            df_laborales.append(tfidf_lab_69d[j])
            labels_laborales.append(display)
            colores_laborales.append(mcolors.to_rgba(base_sat, alpha=0.9))
        fechas_laborales.extend([pd.NaT] * tfidf_lab_69d.shape[0])

df_total = np.vstack(df_academicos + df_laborales)
labels_total = labels_academicos + labels_laborales
colores_total = colores_academicos + colores_laborales

print("Aplicando UMAP...")
umap_model = umap.UMAP(n_components=2, random_state=42)
umap_result_global = umap_model.fit_transform(df_total)

# Similitudes (coseno) normalizadas
from sklearn.preprocessing import normalize
labor_matrix = np.vstack(df_laborales)
labor_norm = normalize(labor_matrix, axis=1)
acad_matrix = np.vstack(df_academicos)
acad_norm = normalize(acad_matrix, axis=1)
sims_por_carrera = [labor_norm.dot(acad_norm[i]) for i in range(len(df_academicos))]

print("✓ UMAP completado")
print(f"  - Total puntos: {len(umap_result_global)}")

Aplicando UMAP...
✓ UMAP completado
  - Total puntos: 64682


In [12]:
# 6. Construcción de figura Plotly con tres trazas por carrera (UMAP1/UMAP2)
fig = go.Figure()

# Cálculo de índices de academicos y laborales en el embedding global
n_acad = len(df_academicos)
acad_coords = umap_result_global[:n_acad]
lab_coords = umap_result_global[n_acad:]

# Índices por carrera en laborales
lab_indices_por_carrera = {}
start = 0
for display, tfidf_lab_69d, fechas, acad_name in vectores_laborales_sorted:
    n = tfidf_lab_69d.shape[0]
    lab_indices_por_carrera[display] = list(range(start, start + n))
    start += n

# Colores por fecha para laborales
colores_por_fecha, _ = colors_by_date_intensity(
    (0.1, 0.6, 0.8, 1.0),
    fechas_laborales
)

for i, (display, vector_acad, acad_name) in enumerate(vectores_academicos_sorted):
    # Colores base por carrera
    base_color = COLOR_OVERRIDES.get(acad_name, colores_base[i % len(colores_base)])
    base_rgba = mcolors.to_rgba(saturate_color(base_color, sat_target=0.95), alpha=1.0)

    # Traza académica (estrella)
    fig.add_trace(go.Scatter(
        x=[acad_coords[i, 0]], y=[acad_coords[i, 1]],
        mode='markers', name=f"Académico - {display}",
        marker=dict(symbol='star', size=14, color=f"rgba({int(base_rgba[0]*255)},{int(base_rgba[1]*255)},{int(base_rgba[2]*255)},1.0)"),
        hovertemplate=f"Carrera: {display}<br>UMAP1: %{{x:.3f}}<br>UMAP2: %{{y:.3f}}<extra></extra>",
        visible=True
    ))

    # Traza ofertas propias (círculos)
    propios_idx = lab_indices_por_carrera[display]
    propios_coords = lab_coords[propios_idx]
    propios_colors = [colores_por_fecha[j] for j in propios_idx]
    fig.add_trace(go.Scatter(
        x=propios_coords[:, 0], y=propios_coords[:, 1],
        mode='markers', name=f"Ofertas propias - {display}",
        marker=dict(symbol='circle', size=8, opacity=1.0, color=propios_colors),
        hovertemplate="Oferta propia<br>UMAP1: %{x:.3f}<br>UMAP2: %{y:.3f}<extra></extra>",
        visible=False
    ))

    # Traza ofertas similares (diamantes, >= 60% y excluye propias)
    sims = sims_por_carrera[i]
    similares_idx_global = [n_acad + k for k in range(len(df_laborales)) if (sims[k] >= 0.60)]
    similares_idx_global_excl = [idx for idx in similares_idx_global if (idx - n_acad) not in propios_idx]
    similares_coords = umap_result_global[similares_idx_global_excl]

    sim_colors = []
    source_labels_sim = []
    for idx in similares_idx_global_excl:
        j = idx - n_acad
        color = colores_por_fecha[j]
        sim_colors.append(color)
        source_labels_sim.append(labels_laborales[j])

    fig.add_trace(go.Scatter(
        x=similares_coords[:, 0], y=similares_coords[:, 1],
        mode='markers', name=f"Ofertas similares 60%+ - {display}",
        marker=dict(symbol='diamond', size=9, opacity=1.0, color=sim_colors),
        customdata=source_labels_sim,
        hovertemplate="Carrera origen: %{customdata}<br>Oferta similar<br>UMAP1: %{x:.3f}<br>UMAP2: %{y:.3f}<extra></extra>",
        visible=False
    ))

# Botones de visibilidad (3 trazas por carrera)
buttons = []
num_carreras = len(vectores_academicos_sorted)
num_traces = num_carreras * 3

# Botón: todas las carreras visibles
buttons.append(dict(
    label='Todas las carreras',
    method='update',
    args=[{'visible': [True] * num_traces}]
))

# Botón por carrera: sólo muestra sus 3 trazas
for i, (display, _, _) in enumerate(vectores_academicos_sorted):
    visible_mask = [False] * num_traces
    visible_mask[i * 3] = True      # académico
    visible_mask[i * 3 + 1] = True  # propias
    visible_mask[i * 3 + 2] = True  # similares
    buttons.append(dict(
        label=display,
        method='update',
        args=[{'visible': visible_mask}]
    ))

fig.update_layout(
    title='UMAP Global - Académicos vs Ofertas',
    xaxis_title='UMAP1', yaxis_title='UMAP2',
    updatemenus=[dict(
        type='dropdown',
        direction='down',
        buttons=buttons,
        x=1.05, xanchor='left',
        y=1.0, yanchor='top'
    )],
    legend=dict(orientation='h', yanchor='bottom', y=-0.08, xanchor='left', x=0.0)
)

pio.write_html(fig, file='UMAP_global.html', auto_open=False)
print("Archivo UMAP_global.html generado correctamente.")

Archivo UMAP_global.html generado correctamente.


In [13]:
# 6. Agregar botones de selección de carrera
buttons = []
for i, carrera in enumerate(nombres_carreras):
    visibility = [False] * len(fig.data)
    base = i * 3
    visibility[base] = True
    visibility[base + 1] = True
    visibility[base + 2] = True
    buttons.append(dict(
        label=carrera,
        method='update',
        args=[{'visible': visibility},
              {'title': f'UMAP: {carrera} - Carrera vs Mercado laboral'}]
    ))
buttons.append(dict(
    label='Todas las carreras',
    method='update',
    args=[{'visible': [True] * len(fig.data)},
          {'title': 'UMAP global: Todas las Carreras vs Mercado laboral'}]
))
print(f"✓ Creados {len(buttons)} botones de selección")

✓ Creados 23 botones de selección


In [14]:
# 7. Configurar layout y exportar a HTML
fig.update_layout(
    title=f'UMAP: {nombres_carreras[0]} - Carrera vs Mercado laboral',
    xaxis_title='UMAP 1',
    yaxis_title='UMAP 2',
    width=1000,
    height=700,
    template='plotly_white',
    hovermode='closest',
    updatemenus=[
        dict(
            buttons=buttons,
            direction='down',
            pad={'r': 10, 't': 10},
            showactive=True,
            x=0.01,
            xanchor='left',
            y=1.15,
            yanchor='top',
            bgcolor='rgba(255, 255, 255, 0.9)',
            bordercolor='#CCCCCC',
            borderwidth=1
        )
    ],
    annotations=[
        dict(
            text='Seleccionar carrera:',
            showarrow=False,
            x=0.01,
            y=1.18,
            xref='paper',
            yref='paper',
            align='left',
            xanchor='left',
            yanchor='top',
            font=dict(size=12, color='#333333')
        )
    ]
)

fig.show()

nombre_archivo = 'UMAP_global.html'
pio.write_html(fig, nombre_archivo, auto_open=False)

print(f"\n{'='*80}")
print(f"✓ Visualización UMAP completada exitosamente")
print(f"{'='*80}")
print(f"\nEstadísticas:")
print(f"   - Carreras analizadas: {len(nombres_carreras)}")
print(f"   - Total de puntos: {len(umap_result_global):,}")
print(f"   - Trazas Plotly: {len(fig.data)}")
print(f"\nArchivo generado: {nombre_archivo}")


✓ Visualización UMAP completada exitosamente

Estadísticas:
   - Carreras analizadas: 22
   - Total de puntos: 64,682
   - Trazas Plotly: 66

Archivo generado: UMAP_global.html
